In [14]:
%cd ~/REVIVAL2
%load_ext autoreload
%autoreload 2
%load_ext blackcellmagic

/disk2/fli/REVIVAL2


In [1]:
import pandas as pd
from scipy.stats import spearmanr

In [3]:
af3_joint = pd.read_csv("/disk2/fli/REVIVAL2/zs/bonddist/af3/struct_joint/PfTrpB-4bromo.csv")
af3_joint.head()

,AAs,fitness,var,n_mut,AA1,AA2,AA3,seq,enzyme,substrate,...,1:GLU-NH_1_2,1:GLU-NH_1_3,1:GLU-NH_1_4,1:GLU-NH_1_agg,2:GLU-NH_2_0,2:GLU-NH_2_1,2:GLU-NH_2_2,2:GLU-NH_2_3,2:GLU-NH_2_4,2:GLU-NH_2_agg
0,AAV,-0.043328,I165A:I183A:Y301V,3,A,A,V,MWFGEFGGQYVPETLVGPLKELEKAYKRFKDDEEFNRQLNYYLKTW...,PfTrpB,4bromo,...,4.132248,2.147623,4.059070,4.133431,2.105471,2.095007,2.101589,4.163435,2.035116,2.095007
1,ADL,0.061846,I165A:I183D:Y301L,3,A,D,L,MWFGEFGGQYVPETLVGPLKELEKAYKRFKDDEEFNRQLNYYLKTW...,PfTrpB,4bromo,...,4.101438,2.021138,3.928323,4.039570,2.607558,2.055020,2.106428,4.068932,1.981890,2.055020
2,AGC,0.005751,I165A:I183G:Y301C,3,A,G,C,MWFGEFGGQYVPETLVGPLKELEKAYKRFKDDEEFNRQLNYYLKTW...,PfTrpB,4bromo,...,4.518341,2.466481,4.357254,4.463213,2.417483,2.407098,2.461830,4.527340,2.326060,2.407098
3,AGL,0.035926,I165A:I183G:Y301L,3,A,G,L,MWFGEFGGQYVPETLVGPLKELEKAYKRFKDDEEFNRQLNYYLKTW...,PfTrpB,4bromo,...,4.155671,2.131715,3.949730,4.039668,2.516566,2.038157,2.150245,4.153675,1.982800,2.038157
4,AMN,-0.002284,I165A:I183M:Y301N,3,A,M,N,MWFGEFGGQYVPETLVGPLKELEKAYKRFKDDEEFNRQLNYYLKTW...,PfTrpB,4bromo,...,3.935914,1.893023,3.880988,3.945079,1.872117,1.871327,1.858211,3.979154,1.809269,1.871327


In [8]:
af3_agg_columns = [c for c in af3_joint.columns if "agg" in c]
af3_agg_columns

['0:C-C_agg', '1:GLU-NH_1_agg', '2:GLU-NH_2_agg']

In [9]:
for c in af3_agg_columns:
    print(c, spearmanr(af3_joint["fitness"], af3_joint[c]))

0:C-C_agg SignificanceResult(statistic=np.float64(-0.24586203090403558), pvalue=np.float64(0.0001151064942909508))
1:GLU-NH_1_agg SignificanceResult(statistic=np.float64(-0.3893119120058903), pvalue=np.float64(3.823141526737016e-10))
2:GLU-NH_2_agg SignificanceResult(statistic=np.float64(-0.41082568630395305), pvalue=np.float64(3.1359204118132095e-11))


In [6]:
chai_joint = pd.read_csv("/disk2/fli/REVIVAL2/zs/bonddist/chai/structure_joint/PfTrpB-4bromo.csv")
chai_joint.head()

,AAs,fitness,var,n_mut,AA1,AA2,AA3,seq,enzyme,substrate,...,1:GLU-NH_1_2,1:GLU-NH_1_3,1:GLU-NH_1_4,1:GLU-NH_1_agg,2:GLU-NH_2_0,2:GLU-NH_2_1,2:GLU-NH_2_2,2:GLU-NH_2_3,2:GLU-NH_2_4,2:GLU-NH_2_agg
0,AAV,-0.043328,I165A:I183A:Y301V,3,A,A,V,MWFGEFGGQYVPETLVGPLKELEKAYKRFKDDEEFNRQLNYYLKTW...,PfTrpB,4bromo,...,7.627141,6.963967,7.264118,6.776257,2.281967,5.723475,5.446602,4.763861,5.118962,4.666973
1,ADL,0.061846,I165A:I183D:Y301L,3,A,D,L,MWFGEFGGQYVPETLVGPLKELEKAYKRFKDDEEFNRQLNYYLKTW...,PfTrpB,4bromo,...,4.735253,4.657432,5.003357,5.313816,5.452778,2.512928,2.526271,2.466567,2.862205,3.164150
2,AGC,0.005751,I165A:I183G:Y301C,3,A,G,C,MWFGEFGGQYVPETLVGPLKELEKAYKRFKDDEEFNRQLNYYLKTW...,PfTrpB,4bromo,...,7.860102,7.460927,7.893424,7.140174,2.351707,6.151737,5.680108,5.275805,5.772861,5.046444
3,AGL,0.035926,I165A:I183G:Y301L,3,A,G,L,MWFGEFGGQYVPETLVGPLKELEKAYKRFKDDEEFNRQLNYYLKTW...,PfTrpB,4bromo,...,8.413980,7.655921,8.018167,7.332324,2.680391,6.496720,6.366911,5.555043,6.455697,5.510952
4,AMN,-0.002284,I165A:I183M:Y301N,3,A,M,N,MWFGEFGGQYVPETLVGPLKELEKAYKRFKDDEEFNRQLNYYLKTW...,PfTrpB,4bromo,...,7.283278,4.517050,6.941712,6.340353,4.850679,3.750977,5.072831,2.308365,4.759011,4.148373


In [11]:
chai_agg_columns = [c for c in chai_joint.columns if "agg" in c]
chai_agg_columns

['0:C-C_agg', '1:GLU-NH_1_agg', '2:GLU-NH_2_agg']

In [12]:
for c in chai_agg_columns:
    print(c, spearmanr(chai_joint["fitness"], chai_joint[c]))

0:C-C_agg SignificanceResult(statistic=np.float64(-0.15619679987492238), pvalue=np.float64(0.015218579499025428))
1:GLU-NH_1_agg SignificanceResult(statistic=np.float64(-0.23068378001001294), pvalue=np.float64(0.000304564994911793))
2:GLU-NH_2_agg SignificanceResult(statistic=np.float64(-0.23219521813739086), pvalue=np.float64(0.0002772121462657627))


In [18]:
import pandas as pd
from glob import glob
from scipy.stats import spearmanr

from REVIVAL.util import get_file_name

for dock_type in ["joint", "seperate"]:

    # Initialize an empty list to store results
    results = []

    # Loop through the CSV files and calculate Spearman correlation
    for lib in sorted(glob(f"/disk2/fli/REVIVAL2/zs/bonddist/chai/structure_{dock_type}/*.csv")):
        lib_name = get_file_name(lib)
        lib_df = pd.read_csv(lib)
        
        df_nan = lib_df[lib_df.isna().any(axis=1)]
        if len(df_nan) > 0:
            print(f"NaN values found in {lib_name}")
            print(df_nan)
        avg_c = [c for c in lib_df.columns if "agg" in c]
        # std_c = [c for c in chai_df.columns if "std" in c]

        # Calculate Spearman correlation for each column in avg_c
        for c in avg_c:
            correlation, p_value = spearmanr(lib_df["fitness"].values, lib_df[c].values)
            # std_col = merged_df[c.replace("avg", "std")].values
            results.append({
                "Library": lib_name,
                "Score_Type": c,
                # "variability": std_col.mean(),
                "Spearman_Correlation": correlation,
                "P_Value": p_value
            })

    # Convert results into a DataFrame and display it as a table
    results_df = pd.DataFrame(results)
    display(results_df)

,Library,Score_Type,Spearman_Correlation,P_Value
0,ParLQ,0:C-C_1_agg,-0.035191,0.437022
1,ParLQ,1:C-C_2_agg,0.010839,0.810859
2,PfTrpB-4bromo,0:C-C_agg,-0.156197,0.015219
3,PfTrpB-4bromo,1:GLU-NH_1_agg,-0.230684,0.000305
4,PfTrpB-4bromo,2:GLU-NH_2_agg,-0.232195,0.000277
5,PfTrpB-4cyano,0:C-C_agg,-0.061533,0.341515
6,PfTrpB-4cyano,1:GLU-NH_1_agg,0.236118,0.000217
7,PfTrpB-4cyano,2:GLU-NH_2_agg,0.230194,0.000314
8,PfTrpB-56chloro,0:C-C_agg,-0.210001,0.001039
9,PfTrpB-56chloro,1:GLU-NH_1_agg,0.024928,0.700207


,Library,Score_Type,Spearman_Correlation,P_Value
0,ParLQ,0:C-C_1_agg,0.169302,0.000166
1,ParLQ,1:C-C_2_agg,0.134358,0.002882
2,Rma-CB,0:C-B_agg,-0.333174,0.000031
3,Rma-CSi,0:C-Si_agg,-0.104207,0.204423


In [22]:
import pandas as pd
from glob import glob
from scipy.stats import spearmanr

from REVIVAL.util import get_file_name

for dock_type in ["joint", "seperate"]:

    # Initialize an empty list to store results
    results = []

    # Loop through the CSV files and calculate Spearman correlation
    for lib in sorted(glob(f"/disk2/fli/REVIVAL2/zs/bonddist/af3/struct_{dock_type}/*.csv")):
        lib_name = get_file_name(lib)
        lib_df = pd.read_csv(lib)
        
        df_nan = lib_df[lib_df.isna().any(axis=1)]
        if len(df_nan) > 0:
            print(f"NaN values found in {lib}")
            print(df_nan)
        avg_c = [c for c in lib_df.columns if "agg" in c]
        # std_c = [c for c in chai_df.columns if "std" in c]

        # Calculate Spearman correlation for each column in avg_c
        for c in avg_c:
            lib_df_valid = lib_df.copy()
            valid_rows = lib_df_valid[["fitness", c]].dropna()
            correlation, p_value = spearmanr(valid_rows["fitness"].values, valid_rows[c].values)
            # correlation, p_value = spearmanr(lib_df["fitness"].values, lib_df[c].values)
            # std_col = merged_df[c.replace("avg", "std")].values
            results.append({
                "Library": lib_name,
                "Score_Type": c,
                # "variability": std_col.mean(),
                "Spearman_Correlation": correlation,
                "P_Value": p_value
            })

    # Convert results into a DataFrame and display it as a table
    results_df = pd.DataFrame(results)
    display(results_df)

NaN values found in /disk2/fli/REVIVAL2/zs/bonddist/af3/struct_joint/ParLQ.csv
       AAs    fitness  selectivity                       var  n_mut AA1 AA2  \
33   WYSQF  10.047669     0.563577                      L59S      1   W   Y   
144  FSFSY  65.233777     5.441362  W56F:Y57S:L59F:Q60S:F89Y      5   F   S   
176  GSVGH   4.593701     0.121676  W56G:Y57S:L59V:Q60G:F89H      5   G   S   
191  HGFAW  38.422778     1.565623  W56H:Y57G:L59F:Q60A:F89W      5   H   G   
211  HPTYG   6.145741     0.188585  W56H:Y57P:L59T:Q60Y:F89G      5   H   P   
247  MEFDY  67.348153    10.698732  W56M:Y57E:L59F:Q60D:F89Y      5   M   E   
304  PASLN   0.736736     0.017550  W56P:Y57A:L59S:Q60L:F89N      5   P   A   
308  PCLTD   0.917851     0.014363       W56P:Y57C:Q60T:F89D      4   P   C   
386  SMSHF  13.900436     0.303771       W56S:Y57M:L59S:Q60H      4   S   M   
403  SSFNY  70.076065     4.271536  W56S:Y57S:L59F:Q60N:F89Y      5   S   S   
462  VSMFT  16.055253     0.421147  W56V:Y57S:L59M:Q

,Library,Score_Type,Spearman_Correlation,P_Value
0,ParLQ,0:C-C_1_agg,-0.292368,6.791560e-11
1,ParLQ,1:C-C_2_agg,-0.280523,4.119975e-10
2,PfTrpB-4bromo,0:C-C_agg,-0.245862,1.151065e-04
3,PfTrpB-4bromo,1:GLU-NH_1_agg,-0.389312,3.823142e-10
4,PfTrpB-4bromo,2:GLU-NH_2_agg,-0.410826,3.135920e-11
5,PfTrpB-4cyano,0:C-C_agg,-0.085907,1.837968e-01
6,PfTrpB-4cyano,1:GLU-NH_1_agg,-0.024102,7.096855e-01
7,PfTrpB-4cyano,2:GLU-NH_2_agg,-0.162945,1.129636e-02
8,PfTrpB-56chloro,0:C-C_agg,-0.056080,3.860810e-01
9,PfTrpB-56chloro,1:GLU-NH_1_agg,-0.052148,4.202997e-01


NaN values found in /disk2/fli/REVIVAL2/zs/bonddist/af3/struct_seperate/ParLQ.csv
       AAs    fitness  selectivity                       var  n_mut AA1 AA2  \
296  NMVDC   6.010976     0.243060  W56N:Y57M:L59V:Q60D:F89C      5   N   M   
297  NMVDV   6.128611     0.313192  W56N:Y57M:L59V:Q60D:F89V      5   N   M   
298  NNVEF  10.406167     0.400124       W56N:Y57N:L59V:Q60E      4   N   N   
299  NNVEQ   2.095577     0.061477  W56N:Y57N:L59V:Q60E:F89Q      5   N   N   
307  PCLTA   2.158520     0.084211       W56P:Y57C:Q60T:F89A      4   P   C   
308  PCLTD   0.917851     0.014363       W56P:Y57C:Q60T:F89D      4   P   C   
309  PCLTH   1.377055     0.144580       W56P:Y57C:Q60T:F89H      4   P   C   
326  PTGEL   2.126261     0.060992  W56P:Y57T:L59G:Q60E:F89L      5   P   T   
328  PTGER   4.675996     0.120611  W56P:Y57T:L59G:Q60E:F89R      5   P   T   
329  PTGES   2.256459     0.039850  W56P:Y57T:L59G:Q60E:F89S      5   P   T   
373  SAFRY  70.447102     4.014563  W56S:Y57A:L59

,Library,Score_Type,Spearman_Correlation,P_Value
0,ParLQ,0:C-C_1_agg,-0.169792,0.000195
1,ParLQ,1:C-C_2_agg,-0.141933,0.001887
2,Rma-CB,0:C-B_agg,-0.009152,0.916417
3,Rma-CSi,0:C-Si_agg,0.069546,0.422841
